In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

base_model = "carloszansavio/bert-financial-sentiment-analysis"

tokenizer = AutoTokenizer.from_pretrained(base_model)

tokenizer = AutoTokenizer.from_pretrained(base_model, device_map="auto")
model = AutoModelForSequenceClassification.from_pretrained(base_model, load_in_8bit=True, torch_dtype=torch.float16)

model = model.eval()

In [2]:
from sklearn.metrics import accuracy_score, f1_score
from datasets import load_dataset
from tqdm import tqdm
import torch
import numpy as np


def test_fiqa(model, tokenizer, batch_size=8):
    dataset = load_dataset('pauri32/fiqa-2018')
    dataset = dataset["test"]
    dataset = dataset.to_pandas()

    dataset["target"] = dataset['label']

    dataset = dataset[['sentence', 'target']]
    dataset.columns = ['input', 'target']

    # print example
    print(f"\n\nPrompt example:\n{dataset['input'][1]}\n\n")

    context = dataset['input'].tolist()
    total_steps = dataset.shape[0]//batch_size + 1
    print(
        f"Total len: {len(context)}. Batchsize: {batch_size}. Total steps: {total_steps}")

    out_text = []

    for i in tqdm(range(total_steps)):
        tmp_context = context[i * batch_size:(i+1) * batch_size]

        tokenizer.pad_token = "[PAD]"

        tokens = tokenizer(tmp_context, return_tensors='pt', padding=True)

        output = model(**tokens)
        output = torch.nn.functional.softmax(output.logits.float(), dim=-1)
        out_text.append(output.detach().numpy())
        torch.cuda.empty_cache()

    out_text = [item for sublist in out_text for item in sublist]
    dataset["out_text"] = out_text
    dataset["new_out"] = dataset["out_text"].apply(np.argmax)

    acc = accuracy_score(dataset["target"], dataset["new_out"])
    f1_macro = f1_score(dataset["target"], dataset["new_out"], average="macro")
    f1_micro = f1_score(dataset["target"], dataset["new_out"], average="micro")
    f1_weighted = f1_score(
        dataset["target"], dataset["new_out"], average="weighted")

    print(f"Acc: {acc}. F1 macro: {f1_macro}. F1 micro: {f1_micro}. F1 weighted (BloombergGPT): {f1_weighted}. ")

    return dataset


dataset = test_fiqa(model, tokenizer)
dataset



Prompt example:
@gakrum nice chart shows distinctive down channel not a dip.. where do you see the bottom? $SPY ..$150? ..$130?


Total len: 150. Batchsize: 8. Total steps: 19


100%|██████████| 19/19 [00:01<00:00, 13.01it/s]

Acc: 0.3333333333333333. F1 macro: 0.21651045559963963. F1 micro: 0.3333333333333333. F1 weighted (BloombergGPT): 0.2165104555996396. 


,input,target,out_text,new_out
0,$HCP Come to the party and buy this -gonna giv...,0,"[0.0042037964, 0.9955279, 0.00026834634]",1
1,@gakrum nice chart shows distinctive down chan...,2,"[0.002397944, 0.997134, 0.00046805132]",1
2,Japan's Asahi to submit bid next week for SABM...,0,"[0.006946154, 0.99237615, 0.0006777612]",1
3,"Tesla Motors recalls 2,700 Model X SUVs $TSLA ...",2,"[0.0034855972, 0.9961649, 0.0003495302]",1
4,CRH's concrete bid for Holcim Lafarge assets,0,"[0.003955714, 0.9957353, 0.00030892907]",1
...,...,...,...,...
145,Intertek swings to ÃÂ£347 mln loss on oil's s...,2,"[0.0006794376, 0.9987244, 0.00059609854]",1
146,RT @jan $ARNA Don't think buyout rumor strong ...,1,"[0.0032405534, 0.9955379, 0.0012215959]",1
147,Barclays appoints JPMorgan's Paul Compton as n...,1,"[0.0013766931, 0.9978689, 0.00075436564]",1
148,$AAPL Now I'm glad I got stopped out of my $11...,1,"[0.0005789791, 0.9988814, 0.0005396678]",1


In [3]:
dataset.to_csv('../../results/financial/bert-sentiment-analysis.csv')

In [4]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
dataset[:20]

,input,target,out_text,new_out
0,$HCP Come to the party and buy this -gonna give solid gains and a dividend $$$$$$,0,"[0.0042037964, 0.9955279, 0.00026834634]",1
1,@gakrum nice chart shows distinctive down channel not a dip.. where do you see the bottom? $SPY ..$150? ..$130?,2,"[0.002397944, 0.997134, 0.00046805132]",1
2,Japan's Asahi to submit bid next week for SABMiller's Grolsch and Peroni - Yomiuri,0,"[0.006946154, 0.99237615, 0.0006777612]",1
3,"Tesla Motors recalls 2,700 Model X SUVs $TSLA https://t.co/F55dx4aegI",2,"[0.0034855972, 0.9961649, 0.0003495302]",1
4,CRH's concrete bid for Holcim Lafarge assets,0,"[0.003955714, 0.9957353, 0.00030892907]",1
5,Doubts grow over GlaxoSmithKline's $6 bln capital return plan,2,"[0.004607267, 0.9948927, 0.0005000395]",1
6,"$XLF http://stks.co/1KJ9 Bounced off target buy area, bought $skf as a hedge. watch 50ma's slope",2,"[0.004453787, 0.99519074, 0.00035538073]",1
7,"$TSLA looking good..i'd love to see a basing action here, letting the RSI take a breather. http://stks.co/g1T7c",0,"[0.003881326, 0.9957937, 0.0003250411]",1
8,France raises concerns over proposed LSE-Deutsche Boerse deal,2,"[0.0010792166, 0.03297234, 0.9659484]",2
9,"Qualcomm: 10% Dividend Increase Rewards Patient Investors, The Future Looks Bright https://t.co/TJjVpqhHqD $QCOM",0,"[0.0014236628, 0.077180974, 0.92139536]",2
